In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)#+(1-(-1)**n)*Delta/2

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)#+(1-(-1)**n)*Delta/2

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*(fermi(T,-t)),bcs(delta_t,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import Slider
%matplotlib qt
def par(n,q0,Ec,delta):
    return Ec*(n-q0)**2+delta*(1-(-1)**n)/2
def find_oc(q0,Ec,delta):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=[e1,e2,e3]
    return a+b.index(min(b))-1

def oc(q0,Ec,delta,T):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=np.array([e1,e2,e3])
    Z=sum(np.exp(-b/T))
    return np.exp(-b/T)/Z,a

def exc(q0,Ec,delta,T,deltat):
    b,a=oc(q0,Ec,delta,T)
    e0=par(a-2,q0,Ec,delta)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    e4=par(a+1,q0,Ec,delta)
    return [b[0],b[0],b[1],b[1],b[2],b[2]],[np.abs(e2-e1)+deltat,-np.abs(e1-e0)-deltat,np.abs(e3-e2)+deltat,-np.abs(e1-e2)-deltat,np.abs(e4-e3)+deltat,-np.abs(e2-e3)-deltat]

fig,ax=plt.subplots(2)

Ec=1
delta=1
q0=1.2
T=0.1
deltat=1
N=range(-5,5)
x=np.linspace(-4,4,100)
for i in N:
    ax[0].plot(x,par(i,x,Ec,delta))

n=find_oc(q0,Ec,delta)
ax[0].scatter(q0,par(n,q0,Ec,delta))
ax[0].set_ylim(0,3)
a,b=exc(q0,Ec,delta,T,deltat)
z=np.zeros(len(x))
for i in range(len(a)):
    z+=a[i]/((x-b[i])**2+0.1)

ax[1].plot(x,z)


axfreq = fig.add_axes([0.25, 0.0, 0.65, 0.03])
q0s = Slider(
    ax=axfreq,
    label='Max',
    valmin=-3,
    valmax=3,
    valinit=0,
)

axfreq = fig.add_axes([0.25, 0.03, 0.65, 0.03])
Ts = Slider(
    ax=axfreq,
    label='Max',
    valmin=0,
    valmax=2,
    valinit=0.1,
)


axamp = fig.add_axes([0.03, 0.25, 0.0225, 0.63])
Ecs = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=2,
    valinit=1,
    orientation="vertical"
)

axamp = fig.add_axes([0.01, 0.25, 0.0225, 0.63])
etas = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=1,
    valinit=0.1,
    orientation="vertical"
)

def update(val):
    ax[0].clear()
    ax[1].clear()
    for i in N:
        ax[0].plot(x,par(i,x,Ecs.val,delta))

    n=find_oc(q0s.val,Ecs.val,delta)
    ax[0].scatter(q0s.val,par(n,q0s.val,Ecs.val,delta))
    ax[0].set_ylim(0,3)
    a,b=exc(q0s.val,Ecs.val,delta,Ts.val,deltat)
    z=np.zeros(len(x))
    for i in range(len(a)):
        z+=a[i]/((x-b[i])**2+etas.val)
    ax[1].plot(x,z)
    

q0s.on_changed(update)
Ecs.on_changed(update) 
Ts.on_changed(update)  
etas.on_changed(update)  


0